# **geemap in python**




### What is Google Earth Engine (GEE)

[Google Earth Engine (GEE)](https://colab.research.google.com/drive/1hs9Pj40Sgg2MkYD7AgNuT7iuuSw04Q0v#scrollTo=Qjqafqd_JBjc&line=7&uniqifier=1) is a cloud-based platform developed by Google for processing and analyzing geospatial data at a planetary scale. It enables users to analyze and visualize vast amounts of satellite imagery and geospatial datasets using a web-based JavaScript API or a Python API. GEE is widely used for environmental and climate research, urban planning, natural resource management, and disaster response, among other fields.

### What is geemap

Geemap is a Python package that extends the functionality of Google Earth Engine (GEE), making it easier for users to interact with GEE data and perform spatial analysis within a Jupyter Notebook or other Python environments

#### Onlnie Learning Source

[geemap online textbook](https://book.geemap.org/chapters/01_introduction.html)

#### Installing geemap

1. Installing with conda
2. Installing with pip
3. Installing from source

#### Earth Engine authentication

1. Create a [google engine account](https://colab.research.google.com/drive/1hs9Pj40Sgg2MkYD7AgNuT7iuuSw04Q0v#scrollTo=Qjqafqd_JBjc&line=24&uniqifier=1)

2. Click 'Get Started to create the project'

3. Go to google cloud console to check your project's information

4. Create secret key with 'ID' of your project

In [ ]:
%pip install -U geemap

In [ ]:
import ee

ee.Authenticate()

### Initialize a project

https://console.cloud.google.com/cloud-resource-manager?walkthrough_id=resource-manager--create-project#step_index=1

Replace the key value for 'ee_project' key in Secrets

In [ ]:
from google.colab import userdata
ee.Initialize(project = userdata.get('ee_project'))

## Load basemap

In [ ]:
import geemap

Map = geemap.Map(center = [42.2602, -71.7980], zoom = 11)
Map

## Metadata about Earth Engine Data

[gee dataset](https://courses.geemap.org/geemap_intro/05_gee_datasets/)

[Earth Engine Data Catelog](https://developers.google.com/earth-engine/datasets)

[ee.ImageCollection](https://developers.google.com/earth-engine/apidocs/ee-imagecollection)

In Google Earth Engine, an ee.ImageCollection is a data structure used to handle a collection of multiple ee.Image objects. It’s often used for time-series analysis, multi-temporal composites, or batch processing of satellite imagery over a specified region

[ee.Image constructor](https://developers.google.com/earth-engine/guides/image_overview#colab-python_1)

In [ ]:
landsat_9 = ee.ImageCollection('LANDSAT/LC09/C02/T1_L2')
landsat_9.size().getInfo()

[aggregate_array](https://developers.google.com/earth-engine/apidocs/ee-imagecollection-aggregate_array)

In [ ]:
landsat_9.limit(10).aggregate_array('system:index').getInfo()

In [ ]:
single_image = ee.Image('LANDSAT/LC09/C02/T1_L2/LC09_001004_20220709').select(
    ['SR_B1', 'SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B7']
)

In [ ]:
geemap.image_props(single_image).getInfo()

## Plot Satellite iamge

[Image Visualization](https://developers.google.com/earth-engine/guides/image_visualization)

Specify the bands combination

[Image.geometry](https://developers.google.com/earth-engine/apidocs/ee-image-geometry)

This list will contain 21 coordinate pairs, each defining a point on the boundary of the image. This data can help you understand the precise extent of the image’s coverage area in geographic space.

In [ ]:
single_image.geometry()

In [ ]:
Map = geemap.Map(center = [78.04207655029894,-10.322619721660091], zoom = 6)

single_image = ee.Image('LANDSAT/LC09/C02/T1_L2/LC09_001004_20220709').select(
    ['SR_B1', 'SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B7']
)

landsat_vis = {'bands': ['SR_B4', 'SR_B3', 'SR_B2'],'gamma': 1.4}

Map.addLayer(single_image, landsat_vis, "Landsat")
Map

## Using the plot tool

[ee.Geometry.Point](https://developers.google.com/earth-engine/apidocs/ee-geometry-point)

Define the center point of your study area in [x,y]

In [ ]:
location = ee.Geometry.Point([-71.80613, 42.27189])

In [ ]:
landsat9_subset = landsat_9.filterBounds(location).filterMetadata('CLOUD_COVER', 'less_than', 30)
landsat9_subset.size().getInfo()

In [ ]:
landsat9_subset.limit(10).aggregate_array('system:index').getInfo()

In [ ]:
Map = geemap.Map(center = [42.27189, -71.80613], zoom = 6)

single_image = ee.Image('LANDSAT/LC09/C02/T1_L2/LC09_012031_20220722').select(
    ['SR_B1', 'SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B7']
)

landsat_vis = {'bands': ['SR_B3', 'SR_B2', 'SR_B1'],'gamma': 1.4}

Map.addLayer(single_image, landsat_vis, "Landsat")
Map

## Visualizing raster data

### Single-band images

Explore the pre-defined tool

In [ ]:
Map = geemap.Map(center=[12, 69], zoom=3)
dem = ee.Image('USGS/SRTMGL1_003')
vis_params = {
    'min': 0,
    'max': 4000,
    'palette': ['006633', 'E5FFCC', '662A00', 'D8D8D8', 'F5F5F5'],
}
Map.addLayer(dem, vis_params, 'SRTM DEM')
Map

## Visualizing vector data

Search TIGER keyword in [data catalog](https://developers.google.com/earth-engine/datasets/catalog)

[Define style of vector data](https://developers.google.com/earth-engine/apidocs/ee-featurecollection-style)

In [ ]:
Map = geemap.Map(center = [42.27189, -71.80613], zoom = 6)
states = ee.FeatureCollection("TIGER/2016/Roads")

vis_params = {
    'color': '0000FFFF',
    'width': 1,
    'lineType': 'solid',
}

Map.addLayer(states.style(**vis_params), {}, "US Roads")
Map

## Creating legends

### Custom legends - vector

In [ ]:
labels = ['US Roads']

# colors can be defined using either hex code or RGB (0-255, 0-255, 0-255)
colors = ['#0000FF']
# legend_colors = [(255, 0, 0), (127, 255, 0), (127, 18, 25), (36, 70, 180), (96, 68 123)]

Map.add_legend(
    labels=labels, colors=colors, position='bottomright'
)
Map

### Built-in legends

In [ ]:
from geemap.legends import builtin_legends

for legend in builtin_legends:
    print(legend)

In [ ]:
Map = geemap.Map(center=[40, -100], zoom=4)
Map.add_basemap('HYBRID')

nlcd = ee.Image('USGS/NLCD_RELEASES/2019_REL/NLCD/2019')
landcover = nlcd.select('landcover')

Map.addLayer(landcover, {}, 'NLCD Land Cover 2019')
Map.add_legend(
    title="NLCD Land Cover Classification", builtin_legend='NLCD', height='465px'
)
Map

### Earth Engine class table

[ESA WorldCover data](https://developers.google.com/earth-engine/datasets/catalog/ESA_WorldCover_v100#description)

In [ ]:
Map = geemap.Map()

dataset = ee.ImageCollection("ESA/WorldCover/v100").first()
Map.addLayer(dataset, {'bands': ['Map']}, "ESA Landcover")

ee_class_table = """
Value	Color	Description
10	006400	Trees
20	ffbb22	Shrubland
30	ffff4c	Grassland
40	f096ff	Cropland
50	fa0000	Built-up
60	b4b4b4	Barren / sparse vegetation
70	f0f0f0	Snow and ice
80	0064c8	Open water
90	0096a0	Herbaceous wetland
95	00cf75	Mangroves
100	fae6a0	Moss and lichen
"""

legend_dict = geemap.legend_from_ee(ee_class_table)
Map.add_legend(title="ESA Land Cover", legend_dict=legend_dict)
Map

## Creating color bars

Adjust palette in tools

In [ ]:
Map = geemap.Map()
dem = ee.Image('USGS/SRTMGL1_003')
vis_params = {
    'min': 0,
    'max': 4000,
    'palette': ['006633', 'E5FFCC', '662A00', 'D8D8D8', 'F5F5F5']
}

Map.addLayer(dem, vis_params, 'SRTM DEM')

Map.add_colorbar(vis_params, label="Elevation (m)", layer_name="SRTM DEM")

Map

## Displaying labels

[US Census States 2016](https://developers.google.com/earth-engine/datasets/catalog/TIGER_2016_States)

In [ ]:
Map = geemap.Map(center=[40, -100], zoom=4, add_google_map=False)
states = ee.FeatureCollection("TIGER/2016/States")
style = {'color': 'black', 'fillColor': "00000000"}
Map.addLayer(states.style(**style), {}, "US States")

Map.add_labels(
    data=states,
    column="NAME",
    font_size="12pt",
    font_color="blue",
    font_family="arial",
    font_weight="bold",
    draggable=True,
)
Map

### Timeseries inspector

In [ ]:
Map = geemap.Map(center=[40, -100], zoom=4)
collection = ee.ImageCollection('USGS/NLCD_RELEASES/2019_REL/NLCD').select('landcover')
vis_params = {'bands': ['landcover']}
years = collection.aggregate_array('system:index').getInfo()
years

In [ ]:
Map.ts_inspector(
    left_ts=collection,
    right_ts=collection,
    left_names=years,
    right_names=years,
    left_vis=vis_params,
    right_vis=vis_params,
    width='80px',
)
Map

#### Practice 1: Create a Time Series Slider

[Data Source](https://developers.google.com/earth-engine/datasets/catalog/MODIS_006_MOD44B)

In [ ]:
Map = geemap.Map(center=[40, -100], zoom=4)
collection = ee.ImageCollection('MODIS/006/MOD44B').select('Percent_Tree_Cover')
vis_params = {
    'min': 0,
    'max': 100,
    'palette': ['bbe029', '0a9501', '074b03']
}
years = collection.aggregate_array('system:index').getInfo()
years

In [ ]:
Map.addLayer(collection, vis_params)
Map.ts_inspector(
    left_ts=collection,
    right_ts=collection,
    left_names=years,
    right_names=years,
    left_vis=vis_params,
    right_vis=vis_params,
    width='80px',
)

Map.add_colorbar(vis_params, label="Percent_Tree_Cover", layer_name="Percent_Tree_Cover")
Map